In [1]:
import rasterio
import numpy as np
import geopandas as gpd
from rasterstats import zonal_stats
import xarray as xr
import pandas as pd
import os



In [5]:
current_dir = os.getcwd()
homedir = os.path.dirname(current_dir)
print("Home Directory:", homedir)

input_dir = os.path.join(homedir, "Input") #this is the input directory  code
output_dir = os.path.join(homedir, "Output") #this is the output directory code
pop_raster_path = os.path.join(input_dir, "GHSL_PAK_2020", "2020_GHS_PAK_POP.tif")
rwi_raster_path = os.path.join(output_dir, "RWI_Raster", "RWI_reproj_resampled.tif")
districts_shapefile_path = os.path.join(input_dir, "admin units", "pak_admbnda_adm2_wfp_20220909.shp")
tehsil_shapefile_path = os.path.join(input_dir, "admin units", "pak_admbnda_adm3_wfp_20220909.shp")



Home Directory: c:\Users\marzi\Downloads\PAK_geospatial_RWI


In [ ]:
# Open the raster dataset
with rasterio.open(pop_raster_path) as dataset:
    # Get the affine transformation matrix
    transform = dataset.transform

    # Get the resolution (pixel size)
    resolution = (transform[0], transform[4])

    # Get the bounds
    bounds = dataset.bounds

print("Resolution (pixel size):", resolution)
print("Bounds (minx, miny, maxx, maxy):", bounds)

# Open the RWI raster dataset
with rasterio.open(rwi_raster_path) as rwi_dataset:
    # Get the affine transformation matrix
    rwi_transform = rwi_dataset.transform

    # Get the resolution (pixel size)
    rwi_resolution = (rwi_transform[0], rwi_transform[4])

    # Get the bounds
    rwi_bounds = rwi_dataset.bounds

print("RWI Raster Resolution (pixel size):", rwi_resolution)
print("RWI Raster Bounds (minx, miny, maxx, maxy):", rwi_bounds)

In [ ]:
# Step 1: Open raster datasets
with rasterio.open(pop_raster_path) as pop_src, rasterio.open(rwi_raster_path) as rwi_src:
    # Read raster bands
    population = pop_src.read(1)
    rwi = rwi_src.read(1)
    # Get the affine transformation matrix
    affine = pop_src.transform

# Define a default CRS (Coordinate Reference System) if needed
default_crs = 'EPSG:54009'

# Step 2: Multiply rasters
population_times_rwi = population * rwi

# Step 3: Zonal statistics by district
districts = gpd.read_file(districts_shapefile_path)

# Ensure CRS compatibility between raster and vector data
population_times_rwi_crs = pop_src.crs.to_dict()
districts = districts.to_crs(population_times_rwi_crs)

# Calculate zonal statistics for districts
district_stats_population_times_rwi = zonal_stats(districts, population_times_rwi, affine=affine, stats=["sum"])
district_stats_population = zonal_stats(districts, population, affine=affine, stats=["sum"])

# Initialize list to store population-weighted RWI
district_population_weighted_rwi = []
# Initialize list to store total population
district_total_population = []

# Loop through each district 
for stat_population_times_rwi, stat_population in zip(district_stats_population_times_rwi, district_stats_population):
    # Total (Population * RWI) in the district
    district_population_times_rwi = stat_population_times_rwi['sum']
    # Total population in the district
    district_population_sum = stat_population['sum']

    # Calculate the population-weighted RWI for the district
    if district_population_sum != 0:
        population_weighted_rwi = district_population_times_rwi / district_population_sum
        district_population_weighted_rwi.append(population_weighted_rwi)
    else:
        # If total population is zero, assign NaN (Not a Number)
        district_population_weighted_rwi.append(np.nan)

    # Append total population for the district
    district_total_population.append(district_population_sum)

# Create a DataFrame to store district IDs and corresponding population-weighted RWI
districts['Population_Weighted_RWI'] = district_population_weighted_rwi
districts['Total_Population'] = district_total_population

columns_to_keep = ['ADM2_EN', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_PCODE', 'Population_Weighted_RWI', 'Total_Population']
districts = districts[columns_to_keep]

csv_file_path = os.path.join(output_dir, "population_weighted_RWI", "P_rwi_district.csv")
districts.to_csv(csv_file_path, index=False)



print("Population-weighted RWI with district information saved to:", csv_file_path)


In [ ]:
# Step 1: Open raster datasets
with rasterio.open(pop_raster_path) as pop_src, rasterio.open(rwi_raster_path) as rwi_src:
    # Read raster bands
    population = pop_src.read(1)
    rwi = rwi_src.read(1)
    # Get the affine transformation matrix
    affine = pop_src.transform

# Define a default CRS (Coordinate Reference System) if needed
default_crs = 'EPSG:54009'

# Step 2: Multiply rasters
population_times_rwi = population * rwi

# Step 3: Zonal statistics by district
tehsil = gpd.read_file(tehsil_shapefile_path)

#  Ensure CRS compatibility between raster and vector data
population_times_rwi_crs = pop_src.crs.to_dict()
tehsil = tehsil.to_crs(population_times_rwi_crs)

# Calculate zonal statistics for districts
tehsil_stats_population_times_rwi = zonal_stats(tehsil, population_times_rwi, affine=affine, stats=["sum"])
tehsil_stats_population = zonal_stats(tehsil, population, affine=affine, stats=["sum"])

# Initialize list to store population-weighted RWI
tehsil_population_weighted_rwi = []
# Initialize list to store total population
tehsil_total_population = []

# Loop through each district and calculate population-weighted RWI
for stat_population_times_rwi, stat_population in zip(tehsil_stats_population_times_rwi, tehsil_stats_population):
    # Total (Population * RWI) in the district
    tehsil_population_times_rwi = stat_population_times_rwi['sum']
    # Total population in the district
    tehsil_population_sum = stat_population['sum']

    # Calculate the population-weighted RWI for the district
    if tehsil_population_sum != 0:
        population_weighted_rwi = tehsil_population_times_rwi / tehsil_population_sum
        tehsil_population_weighted_rwi.append(population_weighted_rwi)
    else:
        # If total population is zero, assign NaN (Not a Number)
        tehsil_population_weighted_rwi.append(np.nan)

    # Append total population for the tehsil
    tehsil_total_population.append(tehsil_population_sum)

# Create a DataFrame to store district IDs and corresponding population-weighted RWI
tehsil['Population_Weighted_RWI'] = tehsil_population_weighted_rwi
tehsil['Total_Population'] = tehsil_total_population

columns_to_keep = ['ADM3_EN', 'ADM3_PCODE','ADM2_EN', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_PCODE', 'Population_Weighted_RWI', 'Total_Population']
tehsil = tehsil[columns_to_keep]

csv_file_path = os.path.join(output_dir, "population_weighted_RWI", "P_rwi_tehsil.csv")
tehsil.to_csv(csv_file_path, index=False)

print("Population-weighted RWI saved to:", csv_file_path)



